# Abstract

The goal of this notebook is to create 57 networks, in which policies have been introduced in Arrondissement 1, (1,2), (1,2,3), 2, (2,3), etc. 
The policies we are testing is a reduction capacity and a reduction in free flow speed. 

Clearer: The policies include:
a. If the free flow speed is > 30, reduce it to 30. -> speed limit
b. Capacity reduction: Reduce the capacity of the streets by half. 


Steps:
1. Load the network and the districts to a gdf.
2. Create 57 networks, in which each 

## 1. Load the original network.xml.gz file

In [12]:
import geopandas as gpd
import pandas as pd
import gzip
import xml.etree.ElementTree as ET

# Function to parse XML and convert to DataFrame
def parse_network_xml_gz(file_path):
    with gzip.open(file_path, 'rb') as f:
        tree = ET.parse(f)
        root = tree.getroot()
        
        data = []
        for element in root.findall(".//node"):
            node_data = {
                'id': element.get('id'),
                'x': float(element.get('x')),
                'y': float(element.get('y'))
            }
            data.append(node_data)
        
        df = pd.DataFrame(data)
        return df

# Load the XML.gz file and convert to GeoDataFrame
file_path = '../../../data/pop_1pm/paris_1pm_network.xml.gz'
df = parse_network_xml_gz(file_path)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y))
gdf_4326 = gdf.set_crs(epsg=2154).to_crs(epsg=4326)

# Display the GeoDataFrame to the user by printing the first few rows
gdf.head()

,id,x,y,geometry
0,1000258241,653283.483192,6.865098e+06,POINT (653283.483 6865098.460)
1,1000258242,653162.315515,6.864742e+06,POINT (653162.316 6864742.138)
2,1000258245,653189.134714,6.864844e+06,POINT (653189.135 6864844.427)
3,1000258255,653177.066854,6.864810e+06,POINT (653177.067 6864810.452)
4,1000258265,653237.165370,6.864983e+06,POINT (653237.165 6864983.374)


In [13]:
districts = gpd.read_file("../../../data/visualisation/districts_paris.geojson")

In [14]:
import geopandas as gpd
from shapely.geometry import LineString

# Function to parse nodes and create a dictionary for quick lookup
def parse_nodes(file_path):
    with gzip.open(file_path, 'rb') as f:
        tree = ET.parse(f)
        root = tree.getroot()
        
        nodes = {}
        nodes_element = root.find("nodes")
        if nodes_element:
            for element in nodes_element.findall("node"):
                node_id = element.attrib['id']
                x, y = float(element.attrib['x']), float(element.attrib['y'])
                nodes[node_id] = (x, y)
        return nodes

# Function to parse edges (links) and add geometry using the nodes' coordinates
def parse_edges(file_path, nodes):
    with gzip.open(file_path, 'rb') as f:
        tree = ET.parse(f)
        root = tree.getroot()
        
        data = []
        links_element = root.find("links")
        if links_element:
            for element in links_element.findall("link"):
                link_data = element.attrib
                from_node = nodes.get(link_data['from'])
                to_node = nodes.get(link_data['to'])
                if from_node and to_node:
                    link_data['geometry'] = LineString([from_node, to_node])
                data.append(link_data)
        df = pd.DataFrame(data)
        return df

# File path to the XML.gz file
file_path = '../../../data/pop_1pm/paris_1pm_network.xml.gz'

# Parse nodes and edges
nodes_dict = parse_nodes(file_path)
df_edges = parse_edges(file_path, nodes_dict)

# Convert to GeoDataFrame and reproject
gdf_edges = gpd.GeoDataFrame(df_edges, geometry='geometry', crs='EPSG:2154')
gdf_edges_4326 = gdf_edges.to_crs(epsg=4326)

# Spatial join to find districts overlapping with each edge
gdf_edges_with_districts = gpd.sjoin(gdf_edges_4326, districts, how='left', op='intersects')

# Group by edge and aggregate the district names
gdf_edges_with_districts = gdf_edges_with_districts.groupby('id').agg({
    'from': 'first',
    'to': 'first',
    'length': 'first',
    'freespeed': 'first',
    'capacity': 'first',
    'permlanes': 'first',
    'oneway': 'first',
    'modes': 'first',
    'geometry': 'first',
    'c_ar': lambda x: list(x.dropna())
}).reset_index()

# Rename the aggregated column to 'district'
gdf_edges_with_districts.rename(columns={'c_ar': 'district'}, inplace=True)

# Display the resulting GeoDataFrame
gdf_edges_with_districts.head()

/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


,id,from,to,length,freespeed,capacity,permlanes,oneway,modes,geometry,district
0,100315,24972409,24972408,16.18125678991161,8.333333333333334,480.0,1.0,1,"bus,car,car_passenger","LINESTRING (2.33869 48.85181, 2.33847 48.85181)",[6.0]
1,100316,5904976363,24983651,14.860209130036054,8.333333333333334,480.0,1.0,1,"bus,car,car_passenger,pt","LINESTRING (2.33874 48.85242, 2.33872 48.85229)",[6.0]
2,100317,24983651,5904976363,14.860209130036054,8.333333333333334,960.0,2.0,1,"bus,car,car_passenger,pt","LINESTRING (2.33872 48.85229, 2.33874 48.85242)",[6.0]
3,100321,664205947,24972376,22.264539971200314,8.333333333333334,960.0,2.0,1,"car,car_passenger","LINESTRING (2.33994 48.85200, 2.33986 48.85181)",[6.0]
4,100324,24972376,24972375,64.85327605625626,8.333333333333334,480.0,1.0,1,"bus,car,car_passenger","LINESTRING (2.33986 48.85181, 2.33909 48.85152)",[6.0]


In [15]:
# # Filter for edges overlapping with at least two districts
# filtered_edges_with_multiple_districts = gdf_edges_with_districts[gdf_edges_with_districts['district'].apply(len) ==0]

# # Convert the filtered DataFrame to a GeoDataFrame
# filtered_gdf_edges_with_multiple_districts = gpd.GeoDataFrame(filtered_edges_with_multiple_districts, geometry='geometry', crs='EPSG:4326')

## 2. Create networks

In [16]:
# Convert freespeed and capacity to numeric values
gdf_edges_with_districts['freespeed'] = pd.to_numeric(gdf_edges_with_districts['freespeed'], errors='coerce')
gdf_edges_with_districts['capacity'] = pd.to_numeric(gdf_edges_with_districts['capacity'], errors='coerce')

In [17]:
# Create 57 dataframes for districts 1, (1,2), (1,2,3), etc.
# Create a list of district combinations
district_combinations = []

# Generate combinations of the form 1, (1,2), (1,2,3), ..., 2, (2,3), (2,3,4), ..., up to 20
for i in range(1, 21):
    district_combinations.append([i])
    if i < 20:
        district_combinations.append([i, i + 1])
    if i < 19:
        district_combinations.append([i, i + 1, i + 2])

## 3. Here we integrate policies in the new dataframes

In [18]:
dataframes = []
for combination in district_combinations:
    df_copy = gdf_edges_with_districts.copy()
    df_copy['policy_introduced'] = df_copy.apply(
        lambda row: 'car' in row['modes'] and any(d in row['district'] for d in combination),
        axis=1
    )    
    # Modify freespeed and capacity based on the policy_introduced condition
    df_copy.loc[df_copy['policy_introduced'], 'freespeed'] = df_copy.loc[df_copy['policy_introduced'], 'freespeed'].apply(lambda x: min(x, 8.3333333333))
    df_copy.loc[df_copy['policy_introduced'], 'capacity'] = df_copy.loc[df_copy['policy_introduced'], 'capacity'] / 20
    
    dataframes.append(df_copy)

In [19]:
import os
import xml.etree.ElementTree as ET
import gzip

# Function to convert a DataFrame back to XML format
def dataframe_to_xml(df, nodes_dict):
    root = ET.Element("network")
    
    # Add attributes
    attributes_elem = ET.SubElement(root, "attributes")
    attribute_dict = {"name": "coordinateReferenceSystem", "class": "java.lang.String"}
    ET.SubElement(attributes_elem, "attribute", attrib=attribute_dict).text = "Atlantis"
    
    # Add nodes
    nodes_elem = ET.SubElement(root, "nodes")
    for node_id, coords in nodes_dict.items():
        ET.SubElement(nodes_elem, "node", id=node_id, x=str(coords[0]), y=str(coords[1]))
    
    # Add links
    links_elem = ET.SubElement(root, "links")
    for _, row in df.iterrows():
        link_attributes = {
            'id': row['id'],
            'from': row['from'],
            'to': row['to'],
            'length': str(row['length']),
            'freespeed': str(row['freespeed']),
            'capacity': str(row['capacity']),
            'permlanes': row['permlanes'],
            'oneway': row['oneway'],
            'modes': row['modes']
        }
        # Replace "inf" with "'Infinity" in the attributes
        for key, value in link_attributes.items():
            if value == "inf":
                link_attributes[key] = "Infinity"
        
        ET.SubElement(links_elem, "link", **link_attributes)
    
    return ET.ElementTree(root)

# Function to write XML to a compressed .gz file
def write_xml_to_gz(xml_tree, file_path):
    xml_declaration = b'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE network SYSTEM "http://www.matsim.org/files/dtd/network_v2.dtd">\n'
    xml_str = xml_declaration + ET.tostring(xml_tree.getroot(), encoding='utf-8', method='xml')
    xml_str = xml_str.replace(b"inf", b"'Infinity")  # Replace "inf" with "'Infinity"
    with gzip.open(file_path, 'wb') as f:
        f.write(xml_str)

# Directory to save the files
output_dir = '../../../data/pop_1pm_policies_new/networks/'
os.makedirs(output_dir, exist_ok=True)

# Create and save the networks
for i, df in enumerate(dataframes):
    # Determine the filename based on the combination
    combination = district_combinations[i]
    if len(combination) == 1:
        filename = f"network_d_{combination[0]}.xml.gz"
    else:
        filename = f"network_d_{'_'.join(map(str, combination))}.xml.gz"
    
    # Convert the DataFrame back to XML
    xml_tree = dataframe_to_xml(df, nodes_dict)
    
    # Write the XML to a compressed .gz file
    file_path = os.path.join(output_dir, filename)
    write_xml_to_gz(xml_tree, file_path)

# Example: Display the file paths of the saved files
output_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir)]
# output_files